<a href="https://colab.research.google.com/github/inforeqd512/QLearning/blob/main/RL_TicTacToe_V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [356]:
import numpy as np
import pylab as plt

# Game

Every simulation starts with a new Game _Board_. 

Each iteration of the simulation ends when the board is full or when any of the players wins.

Every Player _Agent_ is setup with their _Policy Trainer_ and follows espilon greedy policy to learn best actions over several iterations of the simulation.

**measure of success** : The Players become good at playing when the 'Draw' percentages are high.. ie. the first player to go learns to play in the middle or lears to play to Draw

It takes approximately 5000 iterations to get to optimal q-values. At that time both players learn the game well enough to win approx 50% of the time. 

### Actors in the Game


_Board_ is the environment. _State_ is the position of the board after a move from any player. Board also provides the possible set of actions. These are set of open positions on the board on which the player can put it's symbol. 

_Agent_ is the player playing the game. 

- Player playing $X$ will be denoted by symbol $1$
- Player playing $O$ will be denoted by symbol $-1$

Every _Agent_ has its own _Policy Trainer_ that learns the best action it should do in the _Board_ environment. 

The _Policy_Trainer_ follows is the epsilon greedy random policy to choose the next action. Based on this policy it learns the _Quality Action or Q-table_ for each player. Typically the Q-values of Player X are different to those of Player O. 

The _Policy_Trainer_ also needs to update the Q-values after performing the action and seeing the reward, so the _Agent_ delegates the choosing of the action and the performing of the action to it. 


_Action_ to be performed is chosen by the agent on the current state of the board. It is the next available position on the board that the agent can choose from. 

        


### Exploration and Exploitation

Initially we explore more as we learn about the possible moves that can be made. Over multiple iterations we start exploiting what we've learnt so we decay the exploration factor epsilon. 

**Decaying epsilon-greedy**: Does the same as epsilon-greedy, however, the epsilon value starts out near 1, and decays over time according to γ to power of x where x represents the iteration the agent is in.  For γ 0.99 is used 

<sup>Source: [The Effect of the Exploration
Strategy on an Agent’s Performance](https://theses.ubn.ru.nl/bitstream/handle/123456789/5216/Nieuwdorp,%20T._BSc_Thesis_2017.pdf?sequence=1) from Thijs Nieuwdorp</sup>

### Quality of Action 

**Q (dict)** - 

- key = state hash, 
- value = is an array representing 9 spaces in the tic tac toe grid,

Starting from top row of grid, from left to right then next row from left to right etc, based on the grid position sent as input action, the array index is computed
      
eg 
- for 0,1 grid position = 0 * 3 (number of rows and cols in grid) + 1 == 1 so the index of 1.. ie at position 2 in the array
- for 1,1 grid position = 1 * 3 + 1 == 4 so 0,1,2,3,4th position in the array 

beware!!! when using the default initialisation of parameter as {} then python allocates the same memory to both instances of PolicyTrainer. As a result both policies showed the same scores. Also showed win for 1 when should show only for -1.. The best way to find this happening is to initialise the game state to a board where only one player can win, but you'll see that both players get a non-zero Q-value


# Board

In [357]:
class Board:
  """ Class that represents the game board of Tic Tac Toe """

  playerX = 1
  playerO = -1

  def __init__(self, rows=3, cols=3):
    self.rows = rows
    self.cols = cols 
    self.state = np.zeros((self.rows, self.cols), dtype=np.int8)
    # self.state = np.array(
    #               [[0,  0, -1],
    #                [0,  1,  1],
    #                [-1, 1, -1]])
    
  def showBoard(self):
    # p1: x  p2: o
    for i in range(0, self.rows):
        print('-------------')
        out = '| '
        for j in range(0, self.cols):
            if self.state[i, j] == 1:
                token = 'x'
            if self.state[i, j] == -1:
                token = 'o'
            if self.state[i, j] == 0:
                token = ' '
            out += token + ' | '
        print(out)
    print('-------------')


  def checkWinner(self):
    """  return winner symbol, if one exists. 0 if no winner"""

    symbols = np.unique(self.state) #unique values , 0, 1, -1
    symbols = symbols[np.nonzero(symbols)] #remove 0's
    winning_symbol = 0 #no winner yet

    for symbol in symbols:
      #check rows
      row = np.any(np.all(self.state == symbol, axis=1))

      #check cols
      col = np.any(np.all(self.state == symbol, axis=0))

      #check diagonals
      diag1 = np.array([self.state[0,0], self.state[1,1], self.state[2,2]])
      diag1 = np.all(diag1 == symbol)

      diag2 = np.array([self.state[2,0], self.state[1,1], self.state[0,2]])
      diag2 = np.all(diag2 == symbol)

      # Check if state has winner and return winner in that case
      if row or col or diag1 or diag2:
        winning_symbol = symbol
        break
  
    return winning_symbol

  def getAvailablePos(self):
    """  Get state positions that have no value ie zeros """
    return np.argwhere(self.state == 0)

  def isBoardFull(self):
    return (len(self.getAvailablePos()) == 0)

  def checkGameEnded(self):
    """ Check if game has ended by observing if there any possible moves left
    or there is a winner """
    ended = (self.isBoardFull()) or (self.checkWinner() != 0)
    return ended

  def setPosition(self, x, y, symbol):
    """  Set state at position (x,y) with symbol """
    self.state[x,y] = symbol

  def getStateHash(self):
    """  Get hash key of state """
    boardHash = str(self.state.reshape(self.rows * self.cols))
    return boardHash
  
  def performAction(self, action_to_perform, symbol):
    """  Perform the action on current board """
    self.setPosition(action_to_perform[0], action_to_perform[1], symbol)
    return self
      

In [358]:
class Agent:
  """ Class that represents the player 
      symbol is 1 for 'X' or -1 for 'O' 
      policy_trainer is initialised with epsilon greedy set to exploration_probability which is reduced overtime"""

  def __init__(self, symbol, exploration_probability):
    self.symbol = symbol
    self.policy_trainer = PolicyTrainer(exploration_probability=exploration_probability, symbol=symbol)
    return

  def performActionPerPolicy(self, state_hash, possible_actions, current_state):
    """ per the explore and exploitation with current epsilon """
    self.policy_trainer.initQValues(state_hash, possible_actions)
    next_action = self.policy_trainer.chooseAction(state_hash, possible_actions)
    self.policy_trainer.performAction(current_state, next_action)

  def applyFinalResult(self, game_board):
    """
      Gets called after the game has finished. Will update the current Q function based on the game outcome.
    """
    self.policy_trainer.applyFinalResult(game_board)

  def epsilonDecayPerIterations(self, num_iterations):
    """ gradually reduces probabilities per iteration but not completely eliminate it"""
    # Reduce probability to explore during training
    # Do not remove completely 
    
    # Decaying epsilon-greedy: Does the same as epsilon-greedy, however, the epsilon value starts out near 1,
    # and decays over time according to γ to power of x where x represents the iteration the agent is in. 
    # For γ 0.99 is used per https://theses.ubn.ru.nl/bitstream/handle/123456789/5216/Nieuwdorp,%20T._BSc_Thesis_2017.pdf?sequence=1
        
    if self.policy_trainer.exploration_probability > 0.01:
      decay = 0.99 ** num_iterations 
      self.policy_trainer.exploration_probability = decay
      print("self.policy_trainer.exploration_probability :", self.policy_trainer.exploration_probability)

  def updateScore(self):
    """ measures how much is the q-values changing over iterations till finally they settle """
    self.policy_trainer.updateScore()

  def newGame(self):
    self.policy_trainer.newGame()

In [359]:
class PolicyTrainer:
  """
      exploration_probability (float) epsilon greedy value
      learning_rate (float)
      discount_factor (float)
      Q (dict) - {state_hash : array of grid positions}
  """
  def __init__(self, symbol, exploration_probability, learning_rate = 0.9, discount_factor = 0.95, grid_size = 3):
    self.Q = {} #beware!!! when using the initiatlisation value a {} then python allocates the same one to both policies
    self.learning_rate = learning_rate
    self.discount_factor = discount_factor
    print("initialise exploration_probability :", exploration_probability)
    self.exploration_probability = exploration_probability
    self.grid_size = grid_size
    self.scores = []
    self.symbol = symbol
    self.num_times_states_seen = {}
    self.move_history = [] #type: List[(state_hash, action)] keeps a history of all moves done to back propagate the reward
    return

  def getActionIndex(self, current_action):
    """ Returns index in the action array position where the q value should be put """
    idx = self.grid_size * current_action[0] + current_action[1]
    return idx

  def initQValues(self, current_state_hash, possible_actions):
    if current_state_hash not in self.Q:
      self.Q[current_state_hash] = np.full(self.grid_size * self.grid_size, -2.0)
      q_values = self.Q[current_state_hash]
      for action in possible_actions:
        idx = self.getActionIndex(action)
        q_values[idx] = 0.65
      


  def getValueQ(self, current_state_hash, current_action) -> float:
    """ Get the quality value of a given action in a given state,
            returns 0 if the state-action pair has not been seen before and in this case it adds that state into the Q table
            Input is state hash key
            and action as an array of position where the symbol should be put ie [0,1] for top row middle square"""
    idx = self.getActionIndex(current_action)
    if current_state_hash in self.Q:
      q_value = self.Q[current_state_hash][idx]
    return q_value

  def setValueQ(self, current_state_hash, current_action, value):
    """ Set value in Q 
    if this is the first time the state is seen, then the value is the array representing the grid"""
    idx = self.getActionIndex(current_action)
    if current_state_hash in self.Q:
      self.Q[current_state_hash][idx] = value
    else:
      self.Q[current_state_hash] = np.zeros(self.grid_size * self.grid_size)
      self.Q[current_state_hash][idx] = value
    return

  def rewardFunction(self, game_board):
    """ when the chosen action is performed on a state, then we get a new state
    and associated reward from this transition is computed here
    if next state is winner then reward is 1 else -1 if loses to the opponent or 0.5 if draw """
    
    reward = 0 
    winner = game_board.checkWinner() # Returns 0 if there is no winner
    if winner != 0: #winner is when it's 1 or -1
      if winner == self.symbol:
          reward = 1
      else:
          reward = 0
    elif game_board.isBoardFull(): #if the game finishes but there is no winner, then reward is 0.5 for draw
      reward = 0.5

    return reward

  def chooseAction(self, state_hash, possible_actions):
    """ choose action per epsilon greedy explore/exploit policy """
    # #Explore
     #doing exploration here causes large fluctuations in the value functions in the topmost level 0 0 0 0 0 0 0 0 0... 
    if random.random() < 0.01: # self.exploration_probability: 
      action = self.chooseRandomAction(possible_actions)
    else:
      #Exploit
      action = self.chooseBestAction(state_hash, possible_actions)
    return action

  def chooseRandomAction(self, possible_actions):
    """ choose random action from list of possible actions in a state """
    random_idx = np.random.choice(possible_actions.shape[0]) #range from count of possible actions eg 6 actions then [0...5]
    action_pos = possible_actions[random_idx]
    return action_pos

  def chooseBestAction(self, current_state_hash, possible_actions):
    """ Get best action given a set of possible actions in a given state """
    # Pick a random action at first
    random_idx = np.random.choice(possible_actions.shape[0])
    best_action = possible_actions[random_idx]

    # Find action that given largest Q in given state
    maxQ = 0 
    for action in possible_actions:
      tmpQ = self.getValueQ(current_state_hash, action)
      if maxQ < tmpQ:
        maxQ = tmpQ
        best_action = action

    return best_action

  def performAction(self, current_state, next_action):
    """ make the move on the board but defer calculating the Q till the end as 
        only at the end will we know the reward and so the true value of these steps
    """
    self.move_history.append((current_state.getStateHash(), next_action))
    next_state = current_state.performAction(next_action, self.symbol)

  def applyFinalResult(self, game_board):
    """ Implements Q-learning iterative algorithm 
    back propagate the reward to the states that the player sees/moves. 
    Back propagation is required as the player can judge the value of the moves only at the end
    also since the other player is making every alternate view, calculating q value when he makes the move is not possible for the first player
    as the policy of the other player makes that move, and q-learning calculation done at that time only applies to the other player
    and by the time the first player sees the move it does not remember the last move to adjust the q_value of that move wrt the other players move
    easy to do this at the end of all moves
    """
    reward = self.rewardFunction(game_board)

    self.move_history.reverse()

    next_maxQ = None #type: float
    
    for (state_hash, action) in self.move_history:

      q_values = self.Q[state_hash]
      idx = self.getActionIndex(action)
      if next_maxQ == None: #first time in the loop
        q_values[idx] = reward #as this is action that led to terminal state (win lose or draw), it gets the final reward TODO: should I add the reward
      else:
        q_values[idx] = q_values[idx] * (1 - self.learning_rate) + self.learning_rate * (0 + self.discount_factor * next_maxQ) #intermediate rewards is zero

      next_maxQ = max(q_values)

    if self.symbol == 1:
      print("printing for symbol X")
      for (state_hash, _) in self.move_history:
        q_values = self.Q[state_hash]
        print(state_hash, " ", q_values)
      print("final result :", self.symbol, reward, self.move_history)


  def newGame(self):
    """
    Called when a new game is about to start. Reset our internal game state.
    """
    self.move_history = []

  def updateScore(self):
    #compute score
    score = 0
    list1 = list(self.Q.values())
    score = np.sum(list1)
    self.scores.append(score)


# Q-Learning : Training

In [360]:
from tqdm import tqdm
import random

In [361]:
def simulate(playerX, playerO, num_sets = 100, num_games_per_set = 100):

  playerX_wins = []
  playerO_wins = []
  draws = []
  count = []    

  for i in tqdm(range(num_sets)):
    playerX_win, playerO_win, draw = playSet(playerX, playerO, num_games_per_set)
    playerX.epsilonDecayPerIterations(i)
    playerO.epsilonDecayPerIterations(i)
    playerX_wins.append(playerX_win*100.0/num_sets)
    playerO_wins.append(playerO_win*100.0/num_sets)
    draws.append(draw*100.0/num_sets)
    count.append(i*num_sets)

  print("playerX_wins :", playerX_wins)
  print("playerO_wins :", playerO_wins)
  print("draws :", draws)


def playSet(playerX, playerO, num_games_per_set):
  # Counters for wins of each agent and total number of games
  nbr_wins_playerX = 0
  nbr_wins_playerO = 0
  nbr_draw = 0

  for j in range(num_games_per_set):
    # Construct game board and reset player states
    print("\nNew Game")
    game = Board()
    playGame(game, playerX, playerO)

    # Check if there is a winner
    winner = game.checkWinner() # Returns 0 if there is no winner
    if winner == playerX.symbol:
      print("X wins")
      nbr_wins_playerX += 1
    elif winner == playerO.symbol:
      print("O wins")
      nbr_wins_playerO += 1
    else:
      print("Draw game")
      nbr_draw += 1

  return nbr_wins_playerX, nbr_wins_playerO, nbr_draw


def playGame(game_board, playerX, playerO):
  playerX.newGame()
  playerO.newGame()

  # Pick current player
  current_player = playerX

  # play full games in each iteration
  while not game_board.checkGameEnded():
    possible_actions = game_board.getAvailablePos()
    state_hash = game_board.getStateHash()

    current_player.performActionPerPolicy(state_hash, possible_actions, game_board)
    # print(current_player.symbol, " after move")
    # game_board.showBoard()
    
    # Swap player
    if current_player == playerX:
        current_player = playerO
    else:
        current_player = playerX

  playerX.applyFinalResult(game_board)
  playerO.applyFinalResult(game_board)


# Q-Learning : Training

In [ ]:
# Epsilon-greedy 
exploration_probability = 1.0 
#1.0

# Initiatlise players
playerX = Agent(Board.playerX, exploration_probability)
playerO = Agent(Board.playerO, exploration_probability)

simulate(playerX, playerO, 200, 50)
# simulate(playerX, playerO, 1, 1)

In [363]:
# (playerX, playerO) = simulate(10000)

In [364]:
# plt.plot(playerX.policy_trainer.scores)
# plt.show()

In [ ]:
playerX.policy_trainer.Q

In [ ]:
playerO.policy_trainer.Q

### How the episodes settle the Q-values

In [367]:
# plt.plot(playerX.policy_trainer.scores)
# plt.show()

In [368]:
# plt.plot(playerO.policy_trainer.scores)
# plt.show()

### Trained Q-values for Player X


In [369]:
float_formatter = "{:.2f}".format
np.set_printoptions(formatter={'float_kind':float_formatter})

In [370]:
playerX.policy_trainer.Q

{'[0 0 0 0 0 0 0 0 0]': array([0.41, 0.41, 0.41, 0.41, 0.41, 0.41, 0.41, 0.41, 0.41]),
 '[ 1 -1  0  0  0  0  0  0  0]': array([-2.00, -2.00, 0.64, 0.64, 0.85, 0.62, 0.62, 0.64, 0.64]),
 '[ 1 -1  1 -1  0  0  0  0  0]': array([-2.00, -2.00, -2.00, -2.00, 0.91, 0.65, 0.65, 0.65, 0.65]),
 '[ 1 -1  1 -1  1 -1  0  0  0]': array([-2.00, -2.00, -2.00, -2.00, -2.00, -2.00, 1.00, 0.65, 0.65]),
 '[ 1  0 -1  0  0  0  0  0  0]': array([-2.00, 0.62, -2.00, 0.62, 0.62, 0.62, 0.90, 0.62, 0.62]),
 '[ 1  1 -1 -1  0  0  0  0  0]': array([-2.00, -2.00, -2.00, -2.00, 0.64, 0.47, 0.06, 0.65, 0.65]),
 '[ 1  1 -1 -1  1 -1  0  0  0]': array([-2.00, -2.00, -2.00, -2.00, -2.00, -2.00, 0.00, 1.00, 0.65]),
 '[ 1  1 -1 -1  1 -1  1 -1  0]': array([-2.00, -2.00, -2.00, -2.00, -2.00, -2.00, -2.00, -2.00, 1.00]),
 '[ 1  0  0 -1  0  0  0  0  0]': array([-2.00, 0.62, 0.84, -2.00, 0.65, 0.65, 0.65, 0.65, 0.65]),
 '[ 1  1 -1 -1  1  0 -1  0  0]': array([-2.00, -2.00, -2.00, -2.00, -2.00, 0.95, -2.00, 0.65, 0.65]),
 '[ 1  1 

### Trained Q-values for Player O

In [371]:
playerO.policy_trainer.Q

{'[1 0 0 0 0 0 0 0 0]': array([-2.00, 0.16, 0.00, 0.09, 0.43, 0.17, 0.01, 0.11, 0.10]),
 '[ 1 -1  1  0  0  0  0  0  0]': array([-2.00, -2.00, -2.00, 0.62, 0.62, 0.62, 0.62, 0.94, 0.65]),
 '[ 1 -1  1 -1  1  0  0  0  0]': array([-2.00, -2.00, -2.00, -2.00, -2.00, 0.00, 0.00, 0.00, 0.00]),
 '[ 1  1 -1  0  0  0  0  0  0]': array([-2.00, -2.00, -2.00, 0.62, 0.50, 0.90, 0.07, 0.49, 0.62]),
 '[ 1  1 -1 -1  1  0  0  0  0]': array([-2.00, -2.00, -2.00, -2.00, -2.00, 0.00, 0.00, 0.00, 0.00]),
 '[ 1  1 -1 -1  1 -1  1  0  0]': array([-2.00, -2.00, -2.00, -2.00, -2.00, -2.00, -2.00, 0.00, 1.00]),
 '[ 1  1  0 -1  0  0  0  0  0]': array([-2.00, -2.00, 0.87, -2.00, 0.00, 0.00, 0.00, 0.00, 0.00]),
 '[ 1  1 -1 -1  1  1 -1  0  0]': array([-2.00, -2.00, -2.00, -2.00, -2.00, -2.00, -2.00, 0.00, 0.00]),
 '[ 1  1  0  0 -1  0  0  0  0]': array([-2.00, -2.00, 0.45, 0.00, -2.00, 0.00, 0.00, 0.00, 0.00]),
 '[ 1  1 -1  1 -1  0  0  0  0]': array([-2.00, -2.00, -2.00, -2.00, -2.00, 0.00, 1.00, 0.65, 0.65]),
 '[ 1  